# Caso Práctico - ENGIE

*Jose Enrique Zafra Mena*

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl

## Carga y preparación de Datos


In [8]:
facturas_df=pd.read_excel('facturas.xlsx')
mapping_df=pd.read_excel('Mapping.xlsx')

# renombramos los Deals
CTV_df=pd.read_excel('JVLNG_CTV_STOK_MARZO_2024_05_23_14_12.xlsx')
IAC_df=pd.read_excel('MEDG2_IAC_GETRA_MARZO_2024_05_23_14_12.xlsx')
AOC2_df=pd.read_excel('SUGS2_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')
AOC_df=pd.read_excel('SUGST_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')


In [13]:
# verificamos las primeras filas de cada dataframe
print("Facturas AGIkey:")
print(facturas_df.head())

print("\nMapping:")
print(mapping_df.head())

print("\nCTV:")
print(CTV_df.head())

print("\nIAC:")
print(IAC_df.head())

print("\nAOC2:")
print(AOC2_df.head())

print("\nAOC:")
print(AOC_df.head())

Facturas AGIkey:
   NumeroFactura       Emisor       RazonSocialEmisor  DepEmisor     Receptor  \
0     2024264672  ESA86484292         ENAGAS GTS, SAU       60.0  ESB82508441   
1     2024264671  ESA86484292         ENAGAS GTS, SAU       60.0  ESB82508441   
2     2324004231  ESA86484334  ENAGAS TRANSPORTE, SAU        NaN  ESB82508441   
3     2024164592  ESA86484292         ENAGAS GTS, SAU       50.0  ESB82508441   
4     2024164591  ESA86484292         ENAGAS GTS, SAU       50.0  ESB82508441   

  RazonSocialReceptor  DepReceptor FechaFactura    Importe Moneda    Estado  \
0    ENGIE ESPAÑA SLU          NaN       4/3/24  123195.13    EUR  Recibida   
1    ENGIE ESPAÑA SLU          NaN       4/3/24  111963.62    EUR  Recibida   
2    ENGIE ESPAÑA SLU          NaN       6/3/24  503763.47    EUR  Recibida   
3    ENGIE ESPAÑA SLU          NaN       4/3/24    1375.12    EUR  Recibida   
4    ENGIE ESPAÑA SLU          NaN       4/3/24    1787.43    EUR  Recibida   

  FechaEstado FechaRe

In [12]:
# verificamos las columnas de cada dataframe
print("\nColumnas de Facturas:")
print(facturas_df.columns)

print("\nColumnas de Mapping:")
print(mapping_df.columns)

print("\nColumnas de CTV:")
print(CTV_df.columns)

print("\nColumnas de IAC:")
print(IAC_df.columns)

print("\nColumnas de AOC2:")
print(AOC2_df.columns)

print("\nColumnas de AOC:")
print(AOC_df.columns)



Columnas de Facturas:
Index(['NumeroFactura', 'Emisor', 'RazonSocialEmisor', 'DepEmisor', 'Receptor',
       'RazonSocialReceptor', 'DepReceptor', 'FechaFactura', 'Importe',
       'Moneda', 'Estado', 'FechaEstado', 'FechaRegistro', 'Destino',
       'Contrato', 'Origen', 'ServicioFacturado', '¿Verificada por BO?',
       'Observaciones'],
      dtype='object')

Columnas de Mapping:
Index(['Origen', 'Servicio facturado', 'Portfolio', 'Commodity', 'DealType'], dtype='object')

Columnas de CTV:
Index(['Unnamed: 0', 'AggregatedKey', 'ApplicationPeriod', 'BookingSource',
       'BOValidated', 'BOValidationDate', 'BOValidator', 'CancelsDealId',
       'Capacity', 'CapacityFrequency',
       ...
       'ExerciseDate', 'ExerciseType', 'Nature', 'OptionNature', 'OptionType',
       'PeriodInfoCollection', 'Script', 'TemplateId', 'TemplateScript',
       'OrderId'],
      dtype='object', length=108)

Columnas de IAC:
Index(['Unnamed: 0', 'AggregatedKey', 'ApplicationPeriod', 'AuctionType',
   

Vamos a renombrar columnas comunes,

In [14]:
facturas_df.rename(columns={'Origen': 'Origin', 'ServicioFacturado': 'Service'}, inplace=True)
mapping_df.rename(columns={'Origen': 'Origin', 'Servicio facturado': 'Service'}, inplace=True)

## Análisis

### Para las facturas

In [17]:
columns_facturas = set(facturas_df.columns)
columns_mapping = set(mapping_df.columns)


# Crear un set de todos los nombres de columnas presentes en cualquier dataframe
both_columns = columns_facturas | columns_mapping 

# Crear un diccionario para almacenar las columnas de cada dataframe
columnas_dict = {
    'Facturas': columns_facturas,
    'Mapping': columns_mapping,
}
print(columnas_dict)

{'Facturas': {'Moneda', 'RazonSocialEmisor', 'Destino', 'Service', 'Importe', 'FechaFactura', 'DepReceptor', 'DepEmisor', 'RazonSocialReceptor', 'Emisor', 'FechaRegistro', 'Receptor', 'NumeroFactura', 'FechaEstado', 'Origin', 'Estado', '¿Verificada por BO?', 'Observaciones', 'Contrato'}, 'Mapping': {'Portfolio', 'DealType', 'Origin', 'Service', 'Commodity'}}


### Para los deals

In [16]:
columns_df1 = set(CTV_df.columns)
columns_df2 = set(IAC_df.columns)
columns_df3 = set(AOC_df.columns)
columns_df4 = set(AOC2_df.columns)

# Crear un set de todos los nombres de columnas presentes en cualquier dataframe
all_columns = columns_df1 | columns_df2 | columns_df3 | columns_df4

# Crear un diccionario para almacenar las columnas de cada dataframe
columns_dict = {
    'CTV': columns_df1,
    'IAC': columns_df2,
    'AOC': columns_df3,
    'AOC2': columns_df4
}

# Verificar columnas faltantes o adicionales en cada dataframe
for df_name, columns in columns_dict.items():
    missing = all_columns - columns
    additional = columns - all_columns
    if not missing and not additional:
        print(f'{df_name} tiene todas las columnas.')
    else:
        if missing:
            print(f'{df_name} le faltan las columnas: {missing}')
        if additional:
            print(f'{df_name} tiene las columnas adicionales: {additional}')

CTV le faltan las columnas: {'FromDeliveryPoint', 'POValidated', 'OriginalId', 'AuctionType', 'FromDeliveryZone', 'SettlementPremium'}
IAC le faltan las columnas: {'OptionNature', 'OriginalId', 'TemplateId', 'ExerciseDate', 'StrikeCurrencyFactor', 'StrikeCurrency', 'N_1', 'Strike', 'FolderName', 'Periods', 'SettlementFrequency', 'Script', 'PeriodInfoCollection', 'TotalQuantity', 'OrderId', 'AverageCalculationMode', 'OptionType', 'TradeValueDate', 'HasShape', 'DeliveryProfileName', 'ExerciseType', 'StrikeQuantityUnit', 'DeliveryPeriod', 'DeliveryProfileCode', 'Folder', 'TemplateProfile', 'TemplateScript', 'HasDeliveryExceptions'}
AOC le faltan las columnas: {'FromDeliveryPoint', 'POValidated', 'OptionNature', 'OptionType', 'OriginalId', 'TemplateId', 'AuctionType', 'ExerciseDate', 'DeliveryPeriod', 'FromDeliveryZone', 'ExerciseType', 'Nature', 'TemplateScript', 'Script', 'AverageCalculationMode', 'SettlementPremium', 'PeriodInfoCollection', 'OrderId'}
AOC2 le faltan las columnas: {'From

In [18]:
common_columns = columns_df1 & columns_df2 & columns_df3 & columns_df4
print(f'Las columnas comunes a todos los Deals son: {common_columns}')

Las columnas comunes a todos los dataframes son: {'VAI', 'CommodityType', 'Currency', 'BookingSource', 'ExtraVAI', 'Quantity', 'MaturityValueDate', 'LegType', 'Counterparty', 'Premium', 'DeliveryType', 'TasPaymentFrequency', 'VersionUpdateDate', 'AggregatedKey', 'TraderCode', 'DeliveryPeriodName', 'TradeProperties', 'TraderName', 'MidPrice', 'FOValidation', 'MaturityDate', 'LegId', 'PremiumCurrency', 'Flows', 'BOValidator', 'IsVirtualLeg', 'Underlying', 'TradingPlace', 'BOValidated', 'Desk', 'CounterpartyCode', 'NextFixingDate', 'ReplacesDealId', 'Portfolio', 'TradingSystem', 'QuantityUnit', 'ApplicationPeriod', 'SettlementDate', 'Status', 'TradeDate', 'MirrorFOValidation', 'Capacity', 'Interruptible', 'DeliveryPoint', 'StartDeliveryDate', 'PortfolioDivision', 'CancelsDealId', 'MirrorPortFolioId', 'Clearer', 'PortfolioName', 'LastUpdateTime', 'PortfolioGop', 'MirrorTradeId', 'DeliveryZone', 'Unnamed: 0', 'CounterpartyLongName', 'VersionCreationDate', 'SettleCurrency', 'UnwindStatus', '

## Conciliación de los datos

In [19]:
# unimos facturas con mapping usando Origin y Service
facturas_mapping_df = pd.merge(facturas_df, mapping_df, on=['Origin', 'Service'], how='inner')


# unimos facturas_mapping con cada Deal usando Origin y Service
# LO HACEMOS INDIVIDUALMENTE PARA OBTENER TODAS LAS POSIBLES COINCIDENCIAS DE DATOS ESPECÍFICOS EN CADA CONJUNTO
CTV_merged_df = pd.merge(facturas_mapping_df, CTV_df, on='Origin', how='inner')
IAC_merged_df = pd.merge(facturas_mapping_df, IAC_df, on='Origin', how='inner')
AOC2_merged_df = pd.merge(facturas_mapping_df, AOC2_df, on='Origin', how='inner')
AOC_merged_df = pd.merge(facturas_mapping_df, AOC_df, on='Origin', how='inner')

## Propuesta de Valor